In [1]:
from ampel.flags.FlagUtils import FlagUtils
from ampel.flags.TransientFlags import TransientFlags
from ampel.view.DBDocVizualizer import DBDocVizualizer
import json

Cosmetic 

In [2]:
vi = DBDocVizualizer()
def pretty_print(arg):
    vi.markdown_display_json(
        json.dumps(arg, indent=4), 
    )

# Matching flags

Simple case: 

In [3]:
pretty_print(
    FlagUtils.enumflags_to_dbquery(
        TransientFlags.INST_ZTF, 
        {'tranId': 'ZTFabc'}, 
        'alFlags'
    )
) 

```json
{
    "tranId": "ZTFabc",
    "alFlags": 2
}
```

Common case: 

In [4]:
pretty_print(
    FlagUtils.enumflags_to_dbquery(
        TransientFlags.INST_ZTF|TransientFlags.ALERT_IPAC, 
        {'tranId': 'ZTFabc'}, 
        'alFlags'
    )
)

```json
{
    "tranId": "ZTFabc",
    "alFlags": {
        "$all": [
            2,
            6
        ]
    }
}
```

Advanced case: 

In [5]:
pretty_print(
    FlagUtils.enumflags_to_dbquery(
        [TransientFlags.INST_ZTF|TransientFlags.ALERT_IPAC, TransientFlags.INST_OTHER1],
         {'tranId': 'ZTFabc'}, 
        'alFlags'
    )
)

```json
{
    "tranId": "ZTFabc",
    "$or": [
        {
            "alFlags": {
                "$all": [
                    2,
                    6
                ]
            }
        },
        {
            "alFlags": 3
        }
    ]
}
```

# Excluding flags

In [6]:
pretty_print(
    FlagUtils.not_enumflags_to_dbquery(
        TransientFlags.INST_ZTF,
         {'tranId': 'ZTFabc'}, 
        'alFlags'
    )
)

```json
{
    "tranId": "ZTFabc",
    "alFlags": {
        "$ne": 2
    }
}
```

In [7]:
pretty_print(
    FlagUtils.not_enumflags_to_dbquery(
        TransientFlags.INST_ZTF|TransientFlags.ALERT_IPAC,
         {'tranId': 'ZTFabc'}, 
        'alFlags'
    )
)

```json
{
    "tranId": "ZTFabc",
    "alFlags": {
        "$not": {
            "$all": [
                2,
                6
            ]
        }
    }
}
```

In [8]:
pretty_print(
    FlagUtils.not_enumflags_to_dbquery(
        [TransientFlags.INST_ZTF|TransientFlags.ALERT_IPAC, TransientFlags.INST_OTHER1],
         {'tranId': 'ZTFabc'}, 
        'alFlags'
    )
)

```json
{
    "tranId": "ZTFabc",
    "$and": [
        {
            "alFlags": {
                "$not": {
                    "$all": [
                        2,
                        6
                    ]
                }
            }
        },
        {
            "alFlags": {
                "$ne": 3
            }
        }
    ]
}
```

# Matching *and* excluding flags
Order matters!

Please call first `enumflags_to_dbquery` then `not_enumflags_to_dbquery`


### Single flag exclusion


In [9]:
query = {'tranId': 'ZTFabc'}

FlagUtils.enumflags_to_dbquery(
    TransientFlags.INST_ZTF,
    query, 
    'alFlags'
)
    
pretty_print(
    FlagUtils.not_enumflags_to_dbquery(
        TransientFlags.HAS_HU_PPT,
         query, 
        'alFlags'
    )
)

```json
{
    "tranId": "ZTFabc",
    "alFlags": {
        "$eq": 2,
        "$ne": 11
    }
}
```

In [10]:
query = {'tranId': 'ZTFabc'}

FlagUtils.enumflags_to_dbquery(
    TransientFlags.INST_ZTF|TransientFlags.ALERT_IPAC,
    query, 
    'alFlags'
)
    
pretty_print(
    FlagUtils.not_enumflags_to_dbquery(
        TransientFlags.HAS_HU_PPT,
         query, 
        'alFlags'
    )
)

```json
{
    "tranId": "ZTFabc",
    "alFlags": {
        "$all": [
            2,
            6
        ],
        "$ne": 11
    }
}
```

In [11]:
query = {'tranId': 'ZTFabc'}

FlagUtils.enumflags_to_dbquery(
    [TransientFlags.INST_ZTF|TransientFlags.ALERT_IPAC, TransientFlags.INST_OTHER1],
    query, 
    'alFlags'
)
    
pretty_print(
    FlagUtils.not_enumflags_to_dbquery(
        TransientFlags.HAS_HU_PPT,
         query, 
        'alFlags'
    )
)

```json
{
    "tranId": "ZTFabc",
    "$or": [
        {
            "alFlags": {
                "$all": [
                    2,
                    6
                ]
            }
        },
        {
            "alFlags": 3
        }
    ],
    "alFlags": {
        "$ne": 11
    }
}
```

### Combi flag exclusion


In [12]:
query = {'tranId': 'ZTFabc'}

FlagUtils.enumflags_to_dbquery(
    TransientFlags.INST_ZTF,
    query, 
    'alFlags'
)
    
pretty_print(
    FlagUtils.not_enumflags_to_dbquery(
        TransientFlags.HAS_HU_PPT|TransientFlags.HAS_SUPERSEEDED_PPT,
         query, 
        'alFlags'
    )
)

```json
{
    "tranId": "ZTFabc",
    "alFlags": {
        "$eq": 2,
        "$not": {
            "$all": [
                11,
                12
            ]
        }
    }
}
```

In [13]:
query = {'tranId': 'ZTFabc'}

FlagUtils.enumflags_to_dbquery(
    TransientFlags.INST_ZTF|TransientFlags.ALERT_IPAC,
    query, 
    'alFlags'
)
    
pretty_print(
    FlagUtils.not_enumflags_to_dbquery(
        TransientFlags.HAS_HU_PPT|TransientFlags.HAS_SUPERSEEDED_PPT,
         query, 
        'alFlags'
    )
)

```json
{
    "tranId": "ZTFabc",
    "alFlags": {
        "$all": [
            2,
            6
        ],
        "$not": {
            "$all": [
                11,
                12
            ]
        }
    }
}
```

In [14]:
query = {'tranId': 'ZTFabc'}

FlagUtils.enumflags_to_dbquery(
    [TransientFlags.INST_ZTF|TransientFlags.ALERT_IPAC, TransientFlags.INST_OTHER1],
    query, 
    'alFlags'
)
    
pretty_print(
    FlagUtils.not_enumflags_to_dbquery(
        TransientFlags.HAS_HU_PPT|TransientFlags.HAS_SUPERSEEDED_PPT,
         query, 
        'alFlags'
    )
)

```json
{
    "tranId": "ZTFabc",
    "$or": [
        {
            "alFlags": {
                "$all": [
                    2,
                    6
                ]
            }
        },
        {
            "alFlags": 3
        }
    ],
    "alFlags": {
        "$not": {
            "$all": [
                11,
                12
            ]
        }
    }
}
```

### Multi flag exclusion


In [15]:
query = {'tranId': 'ZTFabc'}

FlagUtils.enumflags_to_dbquery(
    TransientFlags.INST_ZTF,
    query, 
    'alFlags'
)
    
pretty_print(
    FlagUtils.not_enumflags_to_dbquery(
        [TransientFlags.HAS_HU_PPT|TransientFlags.HAS_SUPERSEEDED_PPT, TransientFlags.HAS_WZM_PPT],
         query,
        'alFlags'
    )
)

```json
{
    "tranId": "ZTFabc",
    "alFlags": {
        "$eq": 2
    },
    "$and": [
        {
            "alFlags": {
                "$not": {
                    "$all": [
                        11,
                        12
                    ]
                }
            }
        },
        {
            "alFlags": {
                "$ne": 10
            }
        }
    ]
}
```

In [16]:
query = {'tranId': 'ZTFabc'}

FlagUtils.enumflags_to_dbquery(
    TransientFlags.INST_ZTF|TransientFlags.ALERT_IPAC,
    query, 
    'alFlags'
)
    
pretty_print(
    FlagUtils.not_enumflags_to_dbquery(
        [TransientFlags.HAS_HU_PPT|TransientFlags.HAS_SUPERSEEDED_PPT, TransientFlags.HAS_WZM_PPT],
         query,
        'alFlags'
    )
)

```json
{
    "tranId": "ZTFabc",
    "alFlags": {
        "$all": [
            2,
            6
        ]
    },
    "$and": [
        {
            "alFlags": {
                "$not": {
                    "$all": [
                        11,
                        12
                    ]
                }
            }
        },
        {
            "alFlags": {
                "$ne": 10
            }
        }
    ]
}
```

In [17]:
query = {'tranId': 'ZTFabc'}

FlagUtils.enumflags_to_dbquery(
    [TransientFlags.INST_ZTF|TransientFlags.ALERT_IPAC, TransientFlags.INST_OTHER1],
    query, 
    'alFlags'
)
    
pretty_print(
    FlagUtils.not_enumflags_to_dbquery(
        [TransientFlags.HAS_HU_PPT|TransientFlags.HAS_SUPERSEEDED_PPT, TransientFlags.HAS_WZM_PPT],
         query,
        'alFlags'
    )
)

```json
{
    "tranId": "ZTFabc",
    "$or": [
        {
            "alFlags": {
                "$all": [
                    2,
                    6
                ]
            }
        },
        {
            "alFlags": 3
        }
    ],
    "$and": [
        {
            "alFlags": {
                "$not": {
                    "$all": [
                        11,
                        12
                    ]
                }
            }
        },
        {
            "alFlags": {
                "$ne": 10
            }
        }
    ]
}
```